<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


Falcon 9 first stage will land successfully


Several examples of an unsuccessful landing are shown here:


More specifically, the launch records are stored in a HTML table shown below:


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [8]:
# use requests.get() method with the provided static_url
# URL halaman Wikipedia Falcon 9
wiki_url = "https://en.wikipedia.org/wiki/Falcon_9"

# Mengirim permintaan GET untuk mengambil halaman HTML
response = requests.get(wiki_url)

# Periksa apakah request berhasil
if response.status_code == 200:
    print("Request berhasil ✅")
else:
    print("Request gagal ❌", response.status_code)

Request berhasil ✅


In [9]:
print(response.text[:500])  # Cetak 500 karakter pertama dari halaman

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Create a `BeautifulSoup` object from the HTML `response`


In [10]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
# Buat objek BeautifulSoup dari response HTML
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [11]:
# Use soup.title attribute
# Cetak judul halaman untuk memastikan parsing berhasil
print("Judul Halaman:", soup.title.string)

Judul Halaman: Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [12]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Gunakan find_all untuk mengambil semua tabel dari halaman
html_tables = soup.find_all('table')

# Periksa jumlah tabel yang ditemukan
print(f"Jumlah tabel ditemukan: {len(html_tables)}")

Jumlah tabel ditemukan: 25


Starting from the third table is our target table contains the actual launch records.


In [13]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty Mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant Mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure Type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure Material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin Engines
</td>
<td>9 sea level
</td></tr>
<tr>
<td>Propellant
</td>
<td>LOX / RP-1

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [21]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Ambil semua elemen header <th> dari tabel pertama
data_values = []

# Loop melalui setiap baris <tr> dalam tabel
for row in first_launch_table.find_all('tr'):
    cells = row.find_all('td')
    
    if len(cells) == 2:  # Pastikan hanya mengambil baris dengan 2 kolom
        column_name = cells[0].get_text(strip=True)  # Ambil teks kolom pertama
        value = cells[1].get_text(strip=True)  # Ambil teks kolom kedua

        column_names.append(column_name)  # Simpan sebagai nama kolom
        data_values.append(value)  # Simpan sebagai nilai

# Buat DataFrame
df = pd.DataFrame([data_values], columns=column_names)

# Tampilkan DataFrame
df.head()

,Height,Height (with interstage),Diameter,Empty Mass,Propellant Mass,Structure Type,,Structure Material,Landing Legs,,Number of Merlin Engines,Propellant,Thrust at Sea Level,Thrust in Vacuum,Specific Impulse (sea-level),Specific Impulse (vacuum sec),Burn Time,"Ascent Attitude Control – Pitch, Yaw",Ascent Attitude Control – Roll,Coast/Descent Attitude Control
0,41.2 m / 135.2 ft,47.7 m / 156.5 ft,3.7 m / 12 ft,"25,600 kg / 56,423 lb","395,700 kg/ 872,369 lb",LOX tank:monocoque,Fuel tank: skin and stringer,Aluminum lithium skin; aluminum domes,Number: 4,Material:carbon fiber;aluminum honeycomb,9 sea level,LOX / RP-1,"7,607 kN / 1,710,000 lbf","8,227 kN / 1,849,500 lbf",283 sec.,312 sec.,162 sec.,Gimbaled engines,Gimbaled engines,Nitrogen gas thrusters and grid fins


Check the extracted column names


In [22]:
print(column_names)

['Height', 'Height (with interstage)', 'Diameter', 'Empty Mass', 'Propellant Mass', 'Structure Type', '', 'Structure Material', 'Landing Legs', '', 'Number of Merlin Engines', 'Propellant', 'Thrust at Sea Level', 'Thrust in Vacuum', 'Specific Impulse (sea-level)', 'Specific Impulse (vacuum sec)', 'Burn Time', 'Ascent Attitude Control – Pitch, Yaw', 'Ascent Attitude Control – Roll', 'Coast/Descent Attitude Control']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [27]:
# Bersihkan nama kolom dari elemen kosong dan spasi ekstra
column_names = [col.strip() for col in column_names if col.strip()]

# Tampilkan kembali daftar kolom setelah pembersihan
print("Nama kolom setelah dibersihkan:", column_names)

# Buat dictionary dengan kolom yang sudah dibersihkan
launch_dict = dict.fromkeys(column_names, [])

# Hapus kolom yang tidak diinginkan, jika ada
launch_dict.pop('Date and time ( )', None)

# Inisialisasi daftar kosong untuk setiap kolom
for key in launch_dict:
    launch_dict[key] = []

print("Nama kolom yang digunakan:", launch_dict.keys())  # Verifikasi ulang


Nama kolom setelah dibersihkan: ['Height', 'Height (with interstage)', 'Diameter', 'Empty Mass', 'Propellant Mass', 'Structure Type', 'Structure Material', 'Landing Legs', 'Number of Merlin Engines', 'Propellant', 'Thrust at Sea Level', 'Thrust in Vacuum', 'Specific Impulse (sea-level)', 'Specific Impulse (vacuum sec)', 'Burn Time', 'Ascent Attitude Control – Pitch, Yaw', 'Ascent Attitude Control – Roll', 'Coast/Descent Attitude Control']
Nama kolom yang digunakan: dict_keys(['Height', 'Height (with interstage)', 'Diameter', 'Empty Mass', 'Propellant Mass', 'Structure Type', 'Structure Material', 'Landing Legs', 'Number of Merlin Engines', 'Propellant', 'Thrust at Sea Level', 'Thrust in Vacuum', 'Specific Impulse (sea-level)', 'Specific Impulse (vacuum sec)', 'Burn Time', 'Ascent Attitude Control – Pitch, Yaw', 'Ascent Attitude Control – Roll', 'Coast/Descent Attitude Control'])


In [30]:
launch_dict= dict.fromkeys(column_names)

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [33]:
# Inisialisasi dictionary untuk menyimpan data
launch_dict = dict.fromkeys(column_names, [])

# Parsing data dari tabel HTML
extracted_row = 0

# Loop untuk ekstrak tabel dari halaman Wiki
for table_number, table in enumerate(soup.find_all('table', 'wikitable plainrowheaders collapsible')):
    for rows in table.find_all("tr"):
        
        # Cek apakah ini adalah header dengan angka (Nomor Flight)
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()  # Pastikan ini angka
            
        # Jika valid, ekstrak datanya
        if flag:
            extracted_row += 1
            row = rows.find_all('td')

            # Menyimpan data sesuai urutan kolom di tabel Wiki
            datetimelist = row[0].text.strip().split(",")
            date = datetimelist[0].strip()
            time = datetimelist[1].strip() if len(datetimelist) > 1 else None

            booster_version = row[1].a.string if row[1].a else None
            launch_site = row[2].a.string if row[2].a else row[2].text.strip()
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            payload_mass = row[4].text.strip() if row[4] else None
            orbit = row[5].text.strip() if row[5] else None
            customer = row[6].text.strip() if row[6] else None
            launch_outcome = list(row[7].strings)[0].strip() if row[7] else None
            booster_landing = row[8].text.strip() if row[8] else None

            # Menambahkan data ke dalam dictionary
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(booster_version)
            launch_dict['Launch Site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster Landing'].append(booster_landing)

# Konversi dictionary ke DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Simpan ke CSV
df.to_csv('spacex_web_scraped.csv', index=False)

# Tampilkan 5 data pertama
print(df.head())


Empty DataFrame
Columns: [Height, Height (with interstage), Diameter, Empty Mass, Propellant Mass, Structure Type, Structure Material, Landing Legs, Number of Merlin Engines, Propellant, Thrust at Sea Level, Thrust in Vacuum, Specific Impulse (sea-level), Specific Impulse (vacuum sec), Burn Time, Ascent Attitude Control – Pitch, Yaw, Ascent Attitude Control – Roll, Coast/Descent Attitude Control]
Index: []


In [38]:
print(df.columns)

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [39]:
missing_values = df["launchpad"].isnull().sum()
print("Number of missing values in launchpad:", missing_values)

Number of missing values in launchpad: 0


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
